# Ranking Estadão dos Mundiais
Usa a metodologia Elo para descobrir quais foram as melhores seleções da história das Copas do Mundo.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
%matplotlib inline

In [2]:
pd.options.display.max_rows = 1000

## Descrição do modelo

Modelo baseado em `eloratings.net/about`, com adaptações

#### K-constants
- 60 para jogos de Copa do Mundo
- 50 para torneios continentais e intercontinentais principais: Eurocopa, Copa América, Jogos Olímpicos até 1930, Eliminatórias da Copa do Mundo na Europa e na América do Sul
- 30 para torneios continentais e intercontinentais secundários: Copa da Ásia, Copa Africana de Nações, Campeonato da CONCACAF, Copa Ouro, eliminatórias da Euro
- 20 para jogos de eliminatórias dos demais continentes
- 15 para eliminatórias dos demais torneios continentais
- 10 para outros torneios oficiais e amistosos

#### Goal weight
O fator K aumenta de acordo com a diferença de gols.

- Para vitórias por 1 gol, K é multplicado por 1
- Para vitórias por 2 gols, K é multplicado por 1.5
- Para vitórias por 3 gols, K é multiplicado por 1.75
- Para vitórias por 4 ou mais gols, K é multiplicado por 1 + (3/4 + (número de gols - 3/8))


#### Diminuição anual de pontos
- A cada temporada, todos os times perdem 10% dos seus pontos. Isso é feito para evitar que as equipes acumulem pontos indefinidamente e para refletir melhor a força dos times a cada momento. Por exemplo, o Brasil de 2017 não está em posição mais alta apenas porque o time foi bem nos anos 90.

### Referências metodológicas:

https://en.wikipedia.org/wiki/World_Football_Elo_Ratings#Basic_calculation_principles

https://www.kaggle.com/kplauritzen/elo-ratings-in-python

http://www.eloratings.net/about


# Base de dados atualizada até o final de abril

In [3]:
# Base de dados atualizada até 17 de junho
df = pd.read_csv('../data/results.csv') 

In [4]:
# Preenche nans - existem alguns para a cidade do jogo
df = df.fillna('unknown')

### Foram adicionados manualmente os Jogos Olímpicos até 1930
- Até 1930, os Jogos Olímpicos eram o principal campeonato de seleçoões do mundo. Depois da primeira Copa do Mundo, participaram apenas amadores ou times sub-23. Escolhemos colocar na base de dados as partidas de Jogos Olímpicos entre 1908 e 1928 por esse motivo. Nos jogos de 1900 e 1904, o futebol foi apenas um esporte de exibição disputado por clubes locais.
- Nota: **inclui** os jogos do torneio de consolação dos Jogos de 1912

### Removemos torneios que não foram organizados pela FIFA, confederaçoões continentais ou sub-confederaçoões
- A base de dados original tem diversos tipos de torneios, incluindo semi-amadores de nações onde o futebol não é muito desenvolvido e competições amistosas não-oficiais. Eles serão desconsiderados.

In [5]:
# Lista com os torneios que devem ser desconsiderados
to_drop = ['Merdeka Tournament', 'Vietnam Independence Cup', 'Indonesia Tournament', 'GaNEFo', 'Korea Cup',
           'Brazil Independence Cup', 'Kirin Cup', 'Amílcar Cabral Cup', 'Nehru Cup', 'Merlion Cup',
           'Lunar New Year Cup', 'Malta International Tournament', 'Dynasty Cup', 'Jordan International Tournament',
           'King Hassan II Tournament', 'Cyprus International Tournament', 'Dunhill Cup', 'USA Cup', 
           'SKN Football Festival', 'UNIFFAC Cup', 'Millennium Cup', "Prime Minister's Cup", "VFF Cup", "Dragon Cup",
           "ABCS Tournament", "Nile Basin Tnmt.", "Nations Cup", "OSN Cup", "Gulf Cup", "Pan American Championship",
           "Windward Islands Tournament", "Baltic Cup", "Balkan Cup", "Nordic Championship", "Atlantic Cup", 'Rous Cup', 
           'Tournoi de France', 'Mundialito', "South Pacific Games", "King's Cup", "UAE Friendship Tournament",
           "Simba Tnmt.",]

# Mantivemos torneios que consistiam de um amistoso entre duas seleções, o que era freqneunte na América do Sul.
# É o caso da Copa Roca, por exemplo.

In [6]:
df = df[~df.tournament.isin(to_drop)]

#### Abaixo, uma lista das competiçoões consideradas:

In [7]:
df.tournament.unique()

array(['Olympic Games', 'Friendly', 'British Championship', 'Copa Lipton',
       'Copa Newton', 'Copa Premio Honor Argentino',
       'Copa Premio Honor Uruguayo', 'Copa Roca', 'Copa América',
       'Copa Chevallier Boutell', 'International Cup', 'FIFA World Cup',
       'Copa Rio Branco', 'FIFA World Cup qualification',
       'CCCF Championship', 'NAFU Championship', 'Copa Oswaldo Cruz',
       'Copa del Pacífico', "Copa Bernardo O'Higgins",
       'AFC Asian Cup qualification', 'AFC Asian Cup',
       'African Cup of Nations', 'Copa Paz del Chaco',
       'UEFA Euro qualification', 'UEFA Euro',
       'African Cup of Nations qualification', 'Copa Carlos Dittborn',
       'CONCACAF Championship', 'Copa Juan Pinto Durán', 'UAFA Cup',
       'CONCACAF Championship qualification', 'Copa Artigas',
       'Copa Ramón Castilla', 'Oceania Nations Cup', 'CECAFA Cup',
       'Copa Félix Bogado', 'CFU Caribbean Cup qualification',
       'CFU Caribbean Cup', 'West African Cup', 'UDEAC Cup', 

### Criar funções auxiliares

In [8]:
# Cria campo com time vencedor e perdedor
def get_winner(row):
    if row.home_score > row.away_score:
        winner = 'home_team'
        loser = 'away_team'
    elif row.home_score < row.away_score:
        winner = 'away_team'
        loser = 'home_team'
    else:
        winner = 'draw'
        loser = 'draw'
        
    return pd.Series({
        'winner':winner,
        'loser':loser
    })

df = df.join(df.apply(get_winner, axis = 1))

In [9]:
# Cria um campo com objetos datetime
def get_datetime(row):
    datetime = pd.to_datetime(row.date, format="%Y-%m-%d")
    return pd.Series({'datetime':datetime})

df['datetime'] = df.apply(get_datetime, axis=1)

In [10]:
def get_year(row):
    year = row.datetime.year
    return pd.Series({'year':year})

df['year'] = df.apply(get_year, axis=1)

In [11]:
# Calcula a diferença do placar
def get_winning_margin(row):
    if row.winner == 'home_team':
        difference = row.home_score - row.away_score
    elif row.winner == 'away_team':
        difference = row.away_score - row.home_score
    else: # It's a draw
        difference = 0
    return pd.Series({'difference':difference})

df['difference'] = df.apply(get_winning_margin, axis=1)

In [12]:
# Calcula o total de gols da partida
df['total_goals'] = df['home_score'] + df['away_score']

In [13]:
# Calcula a expectativa de vitória
def get_win_expectancy(elo_a, elo_b):
    dr = elo_a - elo_b
    exp_a = 1 / (10**(-dr/400) + 1)
    exp_b = 1 - exp_a
    return (exp_a, exp_b)

In [14]:
# Calcula a variação elo
def get_home_new_elo(result, chances, difference, k_weight, old_elo):
    # PARÂMETROS
    # result: 0, 0.5 ou 1 - derrota, empate ou vitória
    # chances: as chances de vitória calculadas pela função get_win_expectancy na forma de uma tupla
    # difference: a diferença de gols do jogo
    # k_weight: o peso do campeonato

    # Adiciona o peso da margem de vitória/derrota
    if difference < 2:
        goal_weight = 1
    if difference == 2:
        goal_weight = 1.5
    if difference == 3:
        goal_weight = 1.75
    if  difference >= 4:
        goal_weight = 1 + (3/4 + (difference-3)/8) 
    
    k_factor = k_weight * goal_weight
    new_elo = round((old_elo + (k_factor * (result - chances[0]))))
    return new_elo
                         
def get_away_new_elo(result, chances, difference, k_weight, old_elo):
    # Adiciona o peso da margem de vitória/derrota
    if difference < 2:
        goal_weight = 1
    if difference == 2:
        goal_weight = 1.5
    if difference == 3:
        goal_weight = 1.75
    if  difference >= 4:
        goal_weight = 1 + (3/4 + (difference-3)/8) 
    
    k_factor = k_weight * goal_weight
    new_elo = round((old_elo + (k_factor * (result - chances[1]))))
    return new_elo

## Fusões e desmembramentos

- Optamos por discordar da FIFA em alguns aspectos: tratamos as seleções de países que se fundiram e/ou desemembraram como equipes separadas. Assim, a Alemanha, a Alemanha Oriental e a Alemanha Ocidental são times diferentes, assim como URSS e Rússia. Fizemos isso para evitar distorções como no caso russo, onde a seleção herdou os resultados soviéticos em decisão mais política do que desporiva, prejudicando países como Ucrânia e atingindo um resultado inconsistente com os resultados alcançados por sua equipe nacional.

### URSS:

#### Aconteceram partidas entre estados membros da URSS no intervalo em que a seleção soviética existiu. Elas serão desconsideradas, já que não são partidas entre seleções oficiais de estados-nação.

#### Jogos entre 1919 e 1992 serão considerados como jogos da URSS - mesmo jogos disputados sob a bandeira provisória da Comunidade dos Estados Independentes. Estão como Rússia no banco de dados.

In [15]:
# Define datas de início e fim da URSS
ussr_beginning = pd.to_datetime('1922-12-31', format="%Y-%m-%d")
ussr_end =  pd.to_datetime('1992-01-25', format="%Y-%m-%d")

#### Estônia, Letônia e Lituânia: 
- Não existiram como nações soberanas entre 16 de junho 1940 e datas variadas de 1991, quando foram ocupadas pelas URSS. Os jogos dentro desse intervalo serão desconsiderados: eram, geralmente, amistosos entre seleções do regime soviético ou de simpatizantes do bloco.

In [16]:
# Define datas de início da ocupação e da independência de cada estado
occupation_start = pd.to_datetime('1940-06-14', format="%Y-%m-%d")
estonia_independence = pd.to_datetime('1991-08-20', format="%Y-%m-%d")
latvia_independence = pd.to_datetime('1990-05-04', format="%Y-%m-%d")
lithuania_independence = pd.to_datetime('1990-03-11', format="%Y-%m-%d")

In [17]:
# Para cada partida no intervalo de tempo em que a Estônia foi parte da União Soviética, derrubar o país

# Define os index que devem ser derrubados
drop = df[
   ( ( df.home_team=='Estonia') | (df.away_team=="Estonia") ) 
   &
   ( (df.datetime > occupation_start) & (df.datetime < estonia_independence ) )
  ].index

# Derruba com base nos índices
df.drop(drop, inplace=True)

In [18]:
# Para cada partida no intervalo de tempo em que a Letônia foi parte da União Soviética, derrubar o país

# Define os índices que devem ser derrubados
drop = df[
   ( ( df.home_team=='Latvia') | (df.away_team=="Latvia") ) 
   &
   ( (df.datetime > occupation_start) & (df.datetime < latvia_independence ) )
  ].index

# Derruba com base nos índices
df.drop(drop, inplace=True)

In [19]:
# Para cada partida no intervalo de tempo em que a Lituânia foi parte da União Soviética, derrubar o país

# Define os índices que devem ser derrubados
drop = df[
   ( ( df.home_team=='Lithuania') | (df.away_team=="Lithuania") ) 
   &
   ( (df.datetime > occupation_start) & (df.datetime < lithuania_independence ) )
  ].index

# Derruba com base nos índices
df.drop(drop, inplace=True)

#### Ucrânia:
- O país se separa da URSS em 24 da agosto de 1991. Jogos anteriores serão removidos.

In [20]:
# Define a data de independência da Ucrânia
ukraine_independence = pd.to_datetime('1991-08-24', format="%Y-%m-%d")

In [21]:
# Derruba partidas disputadas durante domínio soviético
drop = df[
   ( ( df.home_team=='Ukraine') | (df.away_team=="Ukraine") ) 
   &
   ( (df.datetime > ussr_beginning) & ( df.datetime < ukraine_independence ) )
  ].index

# Derruba com base nos índices
df.drop(drop, inplace=True)

#### Belarus
- Independéncia em 25 de agosto de 1991. Jogos anteriores serão removidos.

In [22]:
belarus_independence = pd.to_datetime('1991-08-25', format="%Y-%m-%d")

In [23]:
# Derruba partidas disputadas durante domínio soviético
drop = df[
   ( ( df.home_team=='Belarus') | (df.away_team=="Belarus") ) 
   &
   ( (df.datetime > ussr_beginning) & ( df.datetime < belarus_independence ) )
  ].index

# Derruba com base nos índices
df.drop(drop, inplace=True)

#### Uzbequistão
- A seleção só existe oficialmente a partir de 17 de junho de 1992. Jogos anteriores serão removidos.

In [24]:
uzbekistan_start = pd.to_datetime('1992-06-17', format="%Y-%m-%d")

In [25]:
drop = df[
   ( ( df.home_team=='Uzbekistan') | (df.away_team=="Uzbekistan") ) 
   &
   ( df.datetime < uzbekistan_start)
   ].index

df.drop(drop, inplace=True)

#### Cazaquistão:
- A seleção só existe oficialmente a partir de primeiro de junho de 1992. Jogos anteriores serão removidos.

In [26]:
kazakhstan_start = pd.to_datetime('1992-06-01', format="%Y-%m-%d")

In [27]:
drop = df[
   ( ( df.home_team=='Kazakhstan') | (df.away_team=="Kazahstan") ) 
   &
   ( df.datetime <  kazakhstan_start)
   ].index

df.drop(drop, inplace=True)

#### Azerbaijão:
- Primeira partida oficial em 17 de setembro de 1992. Partidas anteriores serão desconsideradas.

In [28]:
azerbaijan_start = pd.to_datetime('1992-09-17', format="%Y-%m-%d")

In [29]:
drop = df[
   ( ( df.home_team=='Azerbaijan') | (df.away_team=="Azerbaijan") ) 
   &
   ( df.datetime <  azerbaijan_start)
   ].index

df.drop(drop, inplace=True)

#### Quirguistão:
- Primeira partida oficial em 23 de agosto de 1992. Partidas anteriores serão desconsideradas.

In [30]:
kyrgyzstan_start = pd.to_datetime('1992-08-23', format="%Y-%m-%d")

In [31]:
drop = df[
   ( ( df.home_team=='Kyrgyzstan') | (df.away_team=="Kyrgyzstan") ) 
   &
   ( df.datetime < kyrgyzstan_start)
   ].index

df.drop(drop, inplace=True)

#### Tadjiquistão:
- Começa em 17 de junho de 1992

In [32]:
tajikistan_start = pd.to_datetime('1992-06-17', format="%Y-%m-%d")

In [33]:
drop = df[
   ( ( df.home_team=='Tajikistan') | (df.away_team=="Tajikistan") ) 
   &
   ( df.datetime < tajikistan_start)
   ].index

df.drop(drop, inplace=True)

#### Georgia
- Ainda que a seleção só exista oficialmente depois da independência em 9 de abril de 1991, ela jogou amistosos antes disso. Eles serão desconsiderados.

In [34]:
georgia_start = pd.to_datetime('1991-04-09', format="%Y-%m-%d")

In [35]:
drop = df[
   ( ( df.home_team=='Georgia') | (df.away_team=="Georgia") ) 
   &
   ( df.datetime < georgia_start)
   ].index

df.drop(drop, inplace=True)

#### Moldávia
- Primeira partida oficial em 2 de julho de 1991. Partidas anteriores serão desconsideradas.

In [36]:
moldova_start = pd.to_datetime('1991-07-02', format="%Y-%m-%d")

In [37]:
drop = df[
   ( ( df.home_team=='Moldova') | (df.away_team=="Moldova") ) 
   &
   ( df.datetime < moldova_start)
   ].index

df.drop(drop, inplace=True)

#### Armênia
- Começa em 12 de outubro de 1992

In [38]:
armenia_start = pd.to_datetime('1992-10-12', format="%Y-%m-%d")

In [39]:
drop = df[
   ( ( df.home_team=='Armenia') | (df.away_team=="Armenia") ) 
   &
   ( df.datetime < armenia_start)
   ].index

df.drop(drop, inplace=True)

#### Turcomenistão
- Começa em primeiro de junho de 1992

In [40]:
turkmenistan_start = pd.to_datetime('1992-06-01', format="%Y-%m-%d")

In [41]:
drop = df[
   ( ( df.home_team=='Turkmenistan') | (df.away_team=="Turkmenistan") ) 
   &
   ( df.datetime < turkmenistan_start)
   ].index

df.drop(drop, inplace=True)

#### Rússia
- Jogos de 1912 até 1914 foram do antigo Império Russo - contam como Rússia
- A partir dessa data e até 25 jan. 1992, é a URSS
- De 25 jan. 1992 até 18 jun. 1992, as partidas foram disputadas pela CEI - conta como URSS, pois a base do time era a mesma
- De 1992 em diante, joga a seleção da Rússia

In [42]:
# Para cada partida no intervalo de tempo da União Soviética, substituir Rússia por URSS
to_change = df[
   ( ( df.home_team=='Russia') | (df.away_team=="Russia") ) 
   &
   ( (df.datetime > ussr_beginning) & (df.datetime < ussr_end ) )
  ]

In [43]:
# Transforma jogos da Rússia em URSS
def get_ussr(row):

    # Se o time da casa for a Rússia, trocar por URSS
    if row.home_team == 'Russia':
        home_team = 'Soviet Union'
    else:
        home_team = row.home_team
        
    # Se o time de fora for a Rússia, trocar por URSS
    if row.away_team == 'Russia':
        away_team = 'Soviet Union'
    else:
        away_team = row.away_team
        
    # Retorna uma séria
    return pd.Series({'home_team':home_team,
                     'away_team':away_team})

In [44]:
# Muda com base nos índices e colunas
function_return = to_change.apply(get_ussr, axis=1)

# Substitui ambas as colunas
df.loc[function_return.index.tolist(), 'home_team'] = function_return.home_team
df.loc[function_return.index.tolist(), 'away_team'] = function_return.away_team


### Iugoslávia

Assim como a URSS, desconsideramos as partidas disputadas por combinados de países-membros durante a existência da união, entre 1920 e 1992 - exceção feita à Croácia, que se uniu tardiamente, nos anos 40.

Após o desmembramento, consideramos Iugoslávia, Sérvia e Montenegro e Sérvia times diferentes.

No banco de dados, Sérvia e Montenegro, que jogou como seleção única após a dissolução da Iugoslávia, estão como `Serbia`, porém. Serão separados.

In [45]:
# Define datas de início e fim do time da Iugoslávia
yugoslavia_start = pd.to_datetime('1920-08-28', format="%Y-%m-%d")
yugoslavia_end = pd.to_datetime('1992-03-25', format="%Y-%m-%d")

### Iugoslávia
Os dados da Iugoslávia (pré-separação) já se encontram corretos no banco de dados.

#### Croácia
- Uma seleção croata existiu durante a ocupação alemã. Além disso, ela disputou amistosos durante a existência da Iugoslávia, principalmente contra outros times das nações que formavam o país. Seguindo critério semelhante ao de países da URSS, jogos anteriores à independência croata serão desconsiderados.

In [46]:
# Data do primeiro jogo oficial da atual seleção croata
croatia_start = pd.to_datetime('1990-10-17', format="%Y-%m-%d")

In [47]:
drop = df[
   ( ( df.home_team=='Croatia') | (df.away_team=="Croatia") ) 
   &
   ( ( df.datetime > yugoslavia_start) & ( df.datetime < croatia_start ) )
   ].index

df.drop(drop, inplace=True)

#### Eslovênia
- Assim como no caso da Croácia, serão desconsiderados os jogos disputados durante a existência da Iugoslávia

In [48]:
# Data do primeiro jogo oficial
slovenia_start = pd.to_datetime('1992-06-03', format="%Y-%m-%d")

In [49]:
drop = df[
   ( ( df.home_team=='Slovenia') | (df.away_team=="Slovenia") ) 
   &
   ( ( df.datetime > yugoslavia_start) & ( df.datetime < slovenia_start ) )
   ].index

df.drop(drop, inplace=True)

#### Bósnia e Herzegovina
- Assim como nos casos anteriores, serão desconsiderados os jogos disputados durante a existência da Iugoslávia

In [50]:
# Data do primeiro jogo oficial
bosnia_herzegovina_start = pd.to_datetime('1995-11-30', format="%Y-%m-%d")

In [51]:
drop = df[
   ( ( df.home_team=='Bosnia-Herzegovina') | (df.away_team=="Bosnia-Herzegovina") ) 
   &
   ( ( df.datetime > yugoslavia_start) & ( df.datetime < bosnia_herzegovina_start ) )
   ].index

df.drop(drop, inplace=True)

#### Macedônia
- Assim como nos casos anteriores, serão desconsiderados os jogos disputados durante a existência da Iugoslávia

In [52]:
macedonia_start = pd.to_datetime('1993-10-13', format="%Y-%m-%d")

In [53]:
drop = df[
   ( ( df.home_team=='Macedonia') | (df.away_team=="Macedonia") ) 
   &
   ( ( df.datetime > yugoslavia_start) & ( df.datetime < macedonia_start ) )
   ].index

df.drop(drop, inplace=True)

#### Montenegro
- Assim como nos casos anteriores, serão desconsiderados os jogos disputados durante a existência da Iugoslávia, mas também de Sérvia e Montenegro

In [54]:
montenegro_start = pd.to_datetime('2007-03-24', format="%Y-%m-%d")

In [55]:
drop = df[
   ( ( df.home_team=='Montenegro') | (df.away_team=="Montenegro") ) 
   &
   ( ( df.datetime > yugoslavia_start) & ( df.datetime < montenegro_start ) )
   ].index

df.drop(drop, inplace=True)

#### Sérvia e Montenegro
É o time que sucede imediatamente a Iugoslávia, por cerca de dez anos com o mesmo nome. Entretanto, a nação mudou de nome em 2003, tornando-se Sérvia e Montenegro. Tratamos a Iugoslávia pós-desemembramento e Sérvia e Montenegro como o mesmo time pois eram, de fato, controlados pela mesma federação.

Assim, todos os jogos entre o fim da Iugoslávia, em 1994, e a separação de Sérvia e Montenegro, em 2006, serão considerados como partidas de Sérvia e Montengro.

In [56]:
serbia_and_montenegro_start = yugoslavia_end
serbia_and_montenegro_end = pd.to_datetime('2006-06-22', format="%Y-%m-%d")

In [57]:
to_change = df[
   ( ( df.home_team=='Serbia') | (df.away_team=="Serbia") ) 
   &
   ( ( df.datetime > serbia_and_montenegro_start) & ( df.datetime < serbia_and_montenegro_end ) )
   ]

In [58]:
# Transforma jogos da Servia em Seriva e Montenegro
def get_serbia_and_montenegro(row):

    # Se o time da casa for a Servia, trocar por Servia e Montenegro
    if row.home_team == 'Serbia':
        home_team = 'Serbia and Montenegro'
    else:
        home_team = row.home_team
        country = row.country
        
    # Se o time de fora for a Rússia, trocar por URSS
    if row.away_team == 'Serbia':
        home_team = row.home_team
        away_team = 'Serbia and Montenegro'
    else:
        away_team = row.away_team
        
    # Altera o nome do país da partida para Sérvia e Montenegro, se for o caso
    if row.country == 'Yugoslavia':
        country = 'Serbia and Montenegro'
    else:
        country = row.country
        
    # Retorna uma série
    return pd.Series({'home_team':home_team,
                     'away_team':away_team,
                     'country':country})

In [59]:
# Muda com base nos índices e colunas
function_return = to_change.apply(get_serbia_and_montenegro, axis=1)

# Substitui ambas as colunas
df.loc[function_return.index.tolist(), 'home_team'] = function_return.home_team
df.loc[function_return.index.tolist(), 'away_team'] = function_return.away_team
df.loc[function_return.index.tolist(), 'country'] = function_return.country

#### Sérvia
No banco de dados, todos os jogos após o fim da Iugoslávia original constam como `Serbia`. Já alteramos para que, entre 1994 e 2006, eles sejam considerados como `Serbia and Montenegro`. Depois disso, cada time consta como sua própria seleção, corretamente. 

### Tchecoslováquia
- Consideramos Tchecoslováquia, República Tcheca e Eslováquia como países distintos.

In [60]:
# Define datas de começo e fim da Tchecoslováquia
czechoslovakia_start = pd.to_datetime('1918-10-20', format="%Y-%m-%d")
czechoslovakia_end = pd.to_datetime('1993-11-18', format="%Y-%m-%d")

#### Boêmia
- Existem no banco de dados partidas anteriores à criação da Tchecoslováquia. Elas foram disputadas pela Boêmia, que não era um país independente, mas uma provínica do Império Austro-Húngaro. Serão desconsideradas.

In [61]:
# Derruba os jogos da Boêmia
drop = df[ ( (df.home_team=='Czechoslovakia') | (df.away_team=='Czechoslovakia') )
     &
   ( ( df.datetime < czechoslovakia_start) | (df.datetime > czechoslovakia_end) )
].index

df.drop(drop, inplace=True)

#### República Tcheca
- Existe apenas após o fim do time Tcheco-eslovaco. Jogos anteriores serão transformados em partidas Tchecoslovacas.


In [62]:
# Define datas de começo e fim da República Tcheca
czech_republic_start = pd.to_datetime('1994-02-23', format="%Y-%m-%d")

In [63]:
# Derruba jogos disputados quando a Rep. Tcheca já existia, mas o time ainda representava a Tchecoslováquia.
to_change = df[ ( (df.home_team=='Czech Republic') | (df.away_team=='Czech Republic') )
  &
  ( (df.datetime < czech_republic_start) ) ]

In [64]:
# Transforma jogos da Servia em Seriva e Montenegro
def get_czechoslovakia(row):

    # Se o time da casa for a Servia, trocar por Servia e Montenegro
    if row.home_team == 'Czech Republic':
        home_team = 'Czechoslovakia'
    else:
        home_team = row.home_team
        country = row.country
        
    # Se o time de fora for a Rússia, trocar por URSS
    if row.away_team == 'Czech Republic':
        home_team = row.home_team
        away_team = 'Czechoslovakia'
    else:
        away_team = row.away_team
        
    # Altera o nome do país da partida para Sérvia e Montenegro, se for o caso
    if ( (row.country == 'Czech Republic') | (row.country == 'Slovakia') ):
        country = 'Czechoslovakia'
    else:
        country = row.country
        
    # Retorna uma série
    return pd.Series({'home_team':home_team,
                     'away_team':away_team,
                     'country':country})

In [65]:
# Muda com base nos índices e colunas
function_return = to_change.apply(get_czechoslovakia, axis=1)

# Substitui ambas as colunas
df.loc[function_return.index.tolist(), 'home_team'] = function_return.home_team
df.loc[function_return.index.tolist(), 'away_team'] = function_return.away_team
df.loc[function_return.index.tolist(), 'country'] = function_return.country

#### Eslováquia
- Existe apenas após o fim da Tchecoslováquia. Jogos anteriores serão desconsiderados. Isso inclui jogos disputados pelo time eslovaco durante a ocupação nazista.

In [66]:
drop = df[ ( (df.home_team=='Slovakia') | (df.away_team=='Slovakia') )
  &
  ( (df.datetime < czechoslovakia_end) ) ].index

df.drop(drop, inplace=True)

#### Tchecoslováquia
- Com as alterações anteriores, os jogos restantes da Tchecoslováquia estão corretos.

### Alemanha
- Consideramos a existência de três times: a Alemanha (que representa tanto os períodos unificados do país quanto o time ocidental durante a separação pós segunda-guerra), o Protetorado de Saar (território independente que existiu brevemente depois da separação das Alamanhas) e Alemanha Ocidental. Oficialmente, tanto a Saarlândia quanto a Alemanha Oriental foram absorvidas pela República Federativa Alemã, o que justifica a escolha de manter uma continuidade de resultados entre os times alemães pré-guerra, da Alemanha Ocidental durante a separação e, posteriormente, da Alemanha reunificada. Assim, mantemos o formato da base de dados original.

### Irlanda
- Três times: Irlanda pré-separação, chamada apenas de Irlanda, até 1921
- República da Irlanda e Irlanda do Norte de 1921 em diante

#### Irlanda
- Os jogos disputados até 1924, considerados partidas da Irlanda do Norte pela FIFA e pelo banco de dados, serão considerados jogos da Irlanda.

In [67]:
# Define uma data de fim da seleção da Irlanda unificada
ireland_end = pd.to_datetime('1922-12-08', format="%Y-%m-%d") # Data em que a Irlanda do Norte escolhe manter-se no Reino Unido

In [68]:
to_change = df[ ( (df.home_team=='Northern Ireland') | (df.away_team=='Northern Ireland') ) 
  &
  ( (df.datetime < ireland_end) )]

In [69]:
def get_ireland(row):
    # Se o time da casa for a Servia, trocar por Servia e Montenegro
    if row.home_team == 'Northern Ireland':
        home_team = 'Ireland'
    else:
        home_team = row.home_team
        country = row.country
        
    # Se o time de fora for a Rússia, trocar por URSS
    if row.away_team == 'Northern Ireland':
        home_team = row.home_team
        away_team = 'Ireland'
    else:
        away_team = row.away_team
        
    # Retorna uma série
    return pd.Series({'home_team':home_team,
                     'away_team':away_team})

In [70]:
# Muda com base nos índices e colunas
function_return = to_change.apply(get_ireland, axis=1)
function_return

# Substitui ambas as colunas
df.loc[function_return.index.tolist(), 'home_team'] = function_return.home_team
df.loc[function_return.index.tolist(), 'away_team'] = function_return.away_team

#### República da Irlanda
- Os jogos nomeados como `Ireland` disputados após o desemembramento da Irlanda serão renomeados para `Republic of Ireland`

In [71]:
to_change = df[ ( (df.home_team=='Ireland') | (df.away_team=='Ireland') ) 
  &
  ( (df.datetime > ireland_end) )] 

In [72]:
def get_republic_of_ireland(row):
    if row.home_team == 'Ireland':
        home_team = 'Republic of Ireland'
    else:
        home_team = row.home_team
        country = row.country
        
    if row.away_team == 'Ireland':
        home_team = row.home_team
        away_team = 'Republic of Ireland'
    else:
        away_team = row.away_team
        
    if ( (row.country == 'Ireland') | (row.country == 'Irisih Free State') | (row.country == 'Éire') ):
        country = 'Republic of Ireland'
    else:
        country = row.country    
    
    # Retorna uma série
    return pd.Series({'home_team':home_team,
                     'away_team':away_team,
                     'country':country})

In [73]:
# Muda com base nos índices e colunas
function_return = to_change.apply(get_republic_of_ireland, axis=1)

# Substitui ambas as colunas
df.loc[function_return.index.tolist(), 'home_team'] = function_return.home_team
df.loc[function_return.index.tolist(), 'away_team'] = function_return.away_team
df.loc[function_return.index.tolist(), 'country'] = function_return.country

#### Irlanda do Norte
- Serão considerados os jogos disputados após a separação das Irlandas, apenas. Depois das alteraçoões anteriores, os dados já estão corretos.

### Vietnã
- Durante o período de separação, existiram dois times - o Vietnã do Sul e o Vietnã do Norte, embora esse último não fosse membro da Fifa. Depois da reunificação, o time só voltou a jogar após reformas políticas em 1989. A base de dados está correta, com os times classificados como `Vietnam Republic`, `North Vietnam` e `Vietnam`.

### Iêmen
Dois times existiram entre 1965 e 1990, até a unificação. Os jogos do Iêmen do Norte estão apenas como `Yemen` no período. Estes serão corrigidos.

#### Iêmen do Sul
- Jogos já estão corretos.

#### Iêmen do Norte

In [74]:
yemen_unification = pd.to_datetime('1990-09-08', format="%Y-%m-%d")

In [75]:
to_change  = df[ ( (df.home_team=='Yemen') | (df.away_team=='Yemen') )
    &
   ( (df.datetime < yemen_unification) )
  ]

In [76]:
to_change.head()

,index,date,home_team,away_team,home_score,away_score,tournament,city,country,loser,winner,datetime,year,difference,total_goals
6109,6109,1966-04-02,Syria,Yemen,4,1,UAFA Cup,Baghdad,Iraq,away_team,home_team,1966-04-02,1966,3,5
6114,6114,1966-04-04,Palestine,Yemen,7,0,UAFA Cup,Baghdad,Iraq,away_team,home_team,1966-04-04,1966,7,7
6118,6118,1966-04-06,Libya,Yemen,14,0,UAFA Cup,Baghdad,Iraq,away_team,home_team,1966-04-06,1966,14,14
8711,8711,1973-08-13,Iraq,Yemen,2,0,Friendly,Benghazi,Libya,away_team,home_team,1973-08-13,1973,2,2
8714,8714,1973-08-14,Libya,Yemen,10,0,Friendly,Benghazi,Libya,away_team,home_team,1973-08-14,1973,10,10


In [77]:
def get_north_yemen(row):
    # Se o time da casa for a Servia, trocar por Servia e Montenegro
    if row.home_team == 'Yemen':
        home_team = 'Yemen AR'
    else:
        home_team = row.home_team
        country = row.country
        
    # Se o time de fora for a Rússia, trocar por URSS
    if row.away_team == 'Yemen':
        home_team = row.home_team
        away_team = 'Yemen AR'
    else:
        away_team = row.away_team
        
    # Retorna uma série
    return pd.Series({'home_team':home_team,
                     'away_team':away_team})

In [78]:
# Muda com base nos índices e colunas
function_return = to_change.apply(get_north_yemen, axis=1)

# Substitui ambas as colunas
df.loc[function_return.index.tolist(), 'home_team'] = function_return.home_team
df.loc[function_return.index.tolist(), 'away_team'] = function_return.away_team

#### Iêmen
- Com as alterações anteriores, os jogos do Iêmen estão corretos.

### República Árabe Unida
- Entre 1958 e 1961, foi uma união das equipes de Síria e Egito. No banco de dados, os jogos dessa época estão como `Egpyt`. Serão renomeados. Os demais jogos de Egito e Síria estão corretos.

In [79]:
uar_begins = pd.to_datetime('1958-02-22', format="%Y-%m-%d")
uar_ends = pd.to_datetime('1961-09-28', format="%Y-%m-%d")

In [80]:
to_change = df[ ( (df.home_team=='Egypt') | (df.away_team=='Egypt') ) 
   &
   ( (df.datetime > uar_begins) & (df.datetime < uar_ends) )
  ]

In [81]:
def get_uar(row):
    # Se o time da casa for a Servia, trocar por Servia e Montenegro
    if row.home_team == 'Egypt':
        home_team = 'United Arab Republic'
    else:
        home_team = row.home_team
        country = row.country
        
    # Se o time de fora for a Rússia, trocar por URSS
    if row.away_team == 'Egypt':
        home_team = row.home_team
        away_team = 'United Arab Republic'
    else:
        away_team = row.away_team
        
    # Retorna uma série
    return pd.Series({'home_team':home_team,
                     'away_team':away_team})

In [82]:
# Muda com base nos índices e colunas
function_return = to_change.apply(get_uar, axis=1)

# Substitui ambas as colunas
df.loc[function_return.index.tolist(), 'home_team'] = function_return.home_team
df.loc[function_return.index.tolist(), 'away_team'] = function_return.away_team

### Antilhas Holandesas
- Existiu até 2011, quando os países se desmembraram. Jogos de países membros (Aruba, até 1986; Curaçao, Sint Maarten e Bonaire, até 2011) serão desconsiderados.

#### Curaçao
- Existe oficialmente a partir de 2011. Antes disso, era conhecido como 'Antilhas Holandesas'. Esse nome será aplicado para os jogos anteriores à fundação do país.

In [83]:
curaçao_start = pd.to_datetime('2010-10-10', format="%Y-%m-%d")

In [84]:
to_change = df[ ( (df.home_team=='Curaçao') | (df.away_team=='Curaçao') ) 
   &
   ( (df.datetime < curaçao_start) )
  ]

In [85]:
def get_netherland_antilles(row):
    if row.home_team == 'Curaçao':
        home_team = 'Netherlands Antilles'
    else:
        home_team = row.home_team
        country = row.country

    if row.away_team == 'Curaçao':
        home_team = row.home_team
        away_team = 'Netherlands Antilles'
    else:
        away_team = row.away_team

    if (row.country == 'Curaçao'):
        country = 'Netherlands Antilles'
    else:
        country = row.country    

    # Retorna uma série
    return pd.Series({'home_team':home_team,
                     'away_team':away_team,
                     'country':country})

In [86]:
function_return = to_change.apply(get_netherland_antilles, axis=1)
# Substitui ambas as colunas
df.loc[function_return.index.tolist(), 'home_team'] = function_return.home_team
df.loc[function_return.index.tolist(), 'away_team'] = function_return.away_team
df.loc[function_return.index.tolist(), 'country'] = function_return.country


#### Aruba
- Ignorar jogos anteriores à independência do país

In [87]:
aruba_start = pd.to_datetime('1986-01-01', format="%Y-%m-%d")

In [88]:
drop = df[ ( (df.home_team=='Aruba') | (df.away_team=='Aruba') ) 
   &
   ( (df.datetime < aruba_start) )
  ].index

df.drop(drop, inplace=True)

#### Sint Maarten
- Ignorar jogos anteriores à independência do país

In [89]:
sint_maarten_start = pd.to_datetime('2010-10-10', format="%Y-%m-%d")

In [90]:
drop = df[ ( (df.home_team=='Sint Maarten') | (df.away_team=='Sint Maarten') ) 
   &
   ( (df.datetime < sint_maarten_start) )
  ].index

df.drop(drop, inplace=True)

#### Bonaire
- Ignorar jogos anteriores à independência do país

In [91]:
bonaire_start = pd.to_datetime('2010-10-10', format="%Y-%m-%d")

In [92]:
drop = df[ ( (df.home_team=='Bonaire') | (df.away_team=='Bonaire') ) 
   &
   ( (df.datetime < bonaire_start) )
  ].index

df.drop(drop, inplace=True)

### Palestina
- Vamos considerar três times distintos. Mandato da Palestina, Israel e Palestina.

#### Mandato da Palestina
- Jogos de Israel antes da criação do Estado serão considerados como Mandato da Palestina.

In [93]:
mandatory_palestine_end = pd.to_datetime('1948-05-14', format="%Y-%m-%d")

In [94]:
to_change = df[ ( (df.home_team=='Israel') | (df.away_team=='Israel') ) 
  &
  ( (df.datetime < mandatory_palestine_end) )]

In [95]:
def get_mandatory_palestine(row):
    if row.home_team == 'Israel':
        home_team = 'Mandatory Palestine'
    else:
        home_team = row.home_team
        country = row.country

    if row.away_team == 'Israel':
        home_team = row.home_team
        away_team = 'Mandatory Palestine'
    else:
        away_team = row.away_team

    if (row.country == 'Palestine'):
        country = 'Mandatory Palestine'
    else:
        country = row.country    

    # Retorna uma série
    return pd.Series({'home_team':home_team,
                     'away_team':away_team,
                      'country':country})

In [96]:
# Aplica a função
function_return = to_change.apply(get_mandatory_palestine, axis=1)
# Substitui ambas as colunas
df.loc[function_return.index.tolist(), 'home_team'] = function_return.home_team
df.loc[function_return.index.tolist(), 'away_team'] = function_return.away_team
df.loc[function_return.index.tolist(), 'country'] = function_return.country

### Israel e Palestina
- Estão corretos depois das alteraçoões feitas.

In [97]:
# Seleciona os países com nome 'Burma'
to_change = df[ ( (df.home_team=='Burma') | (df.away_team=='Burma') ) ]

In [98]:
def get_myanmar(row):
    if row.home_team == 'Burma':
        home_team = 'Myanmar'
    else:
        home_team = row.home_team
        country = row.country

    if row.away_team == 'Burma':
        home_team = row.home_team
        away_team = 'Myanmar'
    else:
        away_team = row.away_team

    if (row.country == 'Burma'):
        country = 'Myanmar'
    else:
        country = row.country    

    # Retorna uma série
    return pd.Series({'home_team':home_team,
                     'away_team':away_team,
                      'country':country})

function_return = to_change.apply(get_myanmar, axis=1)
# Substitui ambas as colunas
df.loc[function_return.index.tolist(), 'home_team'] = function_return.home_team
df.loc[function_return.index.tolist(), 'away_team'] = function_return.away_team
df.loc[function_return.index.tolist(), 'country'] = function_return.country

# Derrubar jogos de países que nunca tenham sido membros da FIFA
- Para manter apenas as naçoões relevantes no ranking, vamos derrubar jogos que tenham participação de times que jamais tenham sido membros plenos da Fifa.

In [99]:
# Seleciona países cujos jogos serão ignorados
to_drop = ['Bonaire', 'Cantabria', 'Greenland', 'Guernsey', 'Jersey', 'Manchukuo',
          'Mayotte', 'North Vietnam', 'Palau', 'Monaco', 'Micronesia FS', 'Northern Cyprus',
          'Sint Maarten', 'St Pierre & Miquelon', 'Wallis Islands and Futuna', 'Nauru', 'Niue',
          'Réunion', 'Tuvalu', "French Guiana", "Guadeloupe", "Martinique", "Northern Mariana Islands", 
           "Saint Martin", "Zanzibar", "Tibet", "St-Barthélemy", "French Guyana"]

In [100]:
# Remove as linhas que contenham algum desses times
df = df[(df.home_team.isin(to_drop) == False) & (df.away_team.isin(to_drop) == False)]

# Ajustar peso das partidas
Vamos atribuir pesos diferentes para as competições de acordo com a região e importância do torneio.

In [101]:
# Separa as eliminatórias da Copa nos diferentes continentes
teams_by_confederation = {
    'Asia':['Afghanistan', 'Bahrain', 'Bangladesh', 'Belarus', 'Bhutan', 'Brunei', 'Cambodia', 'China',
           'China PR', 'East Timor', 'Guam', 'Hong Kong', 'India', 'Indonesia', 'Iran', 'Iraq', 'Japan', 
            'Jordan', 'Korea DPR', 'Korea Republic', 'Kuwait', 'Kyrgyzstan', 'Laos', 'Lebanon', 'Macau', 
           'Malaysia', 'Maldives', 'Mandatory Palestine', 'Mongolia', 'Myanmar', 'Nepal', 'Northern Mariana Islands',
           'Oman', 'Pakistan', 'Palestine', 'Philippines', 'Qatar', 'Saudi Arabia', 'Singapore', 'Sri Lanka', 'Taiwan',
           'Tajikistan', 'Thailand', 'Turkmenistan', 'Uzbekistan', 'Vietnam', 'Vietnam Republic', 'Yemen', 'Yemen AR',
           'Yemen DPR', 'United Arab Republic', 'United Arab Emirates', 'Syria'],
    
    
    'Africa':['Algeria', 'Angola', 'Benin', 'Botswana', 'Burkina Faso', 'Burundi', 'Cameroon', 'Cape Verde',
             'Central African Republic', 'Chad', 'Comoros', 'Congo', 'Congo DR', 'Djibouti', 'Egypt', 
              'Equatorial Guinea', 'Eritrea', 'Ethiopia', 'Gabon', 'Gambia', 'Ghana', 'Guinea', 'Guinea-Bissau',
             'Ivory Coast', 'Kenya', 'Lesotho', 'Liberia', 'Libya', 'Madagascar', 'Malawi', 'Mali', 'Mauritania',
             'Mauritius', 'Morocco', 'Mozambique', 'Namibia', 'Niger', 'Nigeria', 'Rwanda', 'Senegal', 'Seychelles',
              'Sierra Leone', 'Somalia', 'South Africa', 'South Sudan', 'Sudan', 'Swaziland', 'São Tomé and Príncipe',
              'Tanzania', 'Togo', 'Tunisia', 'Uganda', 'Zambia', 'Zanzibar', 'Zimbabwe'
             ],
    
    
    'Europe':['Albania','Andorra', 'Armenia', 'Austria', 'Azerbaijan', 'Belgium', 'Bosnia-Herzegovina', 'Bulgaria',
             'Croatia', 'Cyprus', 'Czech Republic', 'Czechoslovakia', 'Denmark', 'England', 'Estonia',
             'Faroe Islands', 'Finland', 'France', 'Georgia', 'German DR', 'Germany', 'Gibraltar',
             'Greece', 'Hungary', 'Iceland', 'Ireland', 'Israel', 'Italy', 'Kazakhstan', 'Kosovo',
              'Latvia', 'Liechtenstein', 'Lithuania', 'Luxembourg', 'Macedonia', 'Malta', 'Moldova', 'Montenegro',
             'Netherlands', 'Northern Ireland', 'Norway', 'Poland', 'Portugal', 'Republic of Ireland', 'Romania',
             'Russia', 'Saarland', 'San Marino', 'Scotland', 'Serbia', 'Serbia and Montenegro', 'Slovakia', 'Slovenia',
             'Soviet Union', 'Spain', 'Sweden', 'Switzerland', 'Turkey', 'Ukraine', 'Wales', 'Yugoslavia'],
    
    
    
    'South America':['Argentina', 'Bolivia', 'Brazil', 'Chile', 'Colombia', 'Ecuador', 'Paraguay', 
                     'Peru', 'Venezuela', 'Uruguay'],
    
    
    
    'North and Central America':['Aruba','Anguilla', 'Antigua and Barbuda', 'Bahamas', 'Barbados', 'Belize', 'Bermuda',
                                'British Virgin Islands', 'Canada', 'Cayman Islands', 'Costa Rica', 'Cuba',
                                'Curaçao', 'Dominica', 'Dominican Republic', 'El Salvador', 'French Guyana',
                                'Grenada', 'Guadeloupe', 'Guyana', 'Haiti', 'Honduras', 'Jamaica', 'Martinique',
                                'Mexico', 'Montserrat', 'Netherlands Antilles','Nicaragua', 'Panama', 'Puerto Rico',
                                'St Kitts and Nevis', 'St Lucia', 'St Martin', 'St Vincent and the Grenadines',
                                'Suriname', 'Trinidad and Tobago', 'Turks and Caicos Islands', 'USA', 'Virgin Islands',
                                'Guatemala'],
    
    
    'Oceania':['American Samoa', 'Australia', 'Cook Islands', 'Fiji', 'Kiribati', 'New Caledonia', 'New Zealand',
              'Papua New Guinea', 'Samoa', 'Solomon Islands', 'Tahiti', 'Tonga', 'Vanuatu'],
}



In [102]:
# Essa função determina em qual continente foi disputada a eliminatória
def get_qualitifcation_continent(row, teams_by_confederation):
    prefix = 'FIFA World Cup qualification: '

    if row.home_team in teams_by_confederation['Asia']:
        tournament = prefix + 'Asia'
    elif row.home_team in teams_by_confederation['Africa']:
        tournament = prefix + 'Africa'
    elif row.home_team in teams_by_confederation['Europe']:
        tournament = prefix + 'Europe'
    elif row.home_team in teams_by_confederation['South America']:
        tournament = prefix + 'South America'
    elif row.home_team in teams_by_confederation['North and Central America']:
        tournament = prefix + 'North and Central America'
    elif row.home_team in teams_by_confederation['Oceania']:
        tournament = prefix + 'Oceania'
        
    return pd.Series({'tournament':tournament})


In [103]:
to_change = df[df.tournament=='FIFA World Cup qualification']
function_return = to_change.apply(get_qualitifcation_continent, args=(teams_by_confederation,), axis=1)
df.loc[function_return.index.tolist(), 'tournament'] = function_return.tournament

In [104]:
# Considerar casos de países que trocaram de federação nas eliminatórias: Austrália, Israel, Cazaquistão, Nova Zelândia.

## Israel
to_change = df[(df.home_team=='Israel') & (df.tournament == "FIFA World Cup qualification: Europe")]
def get_right_israel(row):
    if row.year in range(1954, 1975):
        tournament = "FIFA World Cup qualification: Asia"
    elif row.year in range(1978, 1991):
        tournament = "FIFA World Cup qualification: Oceania"
    else:
        tournament = row.tournament
    
    return pd.Series({'tournament':tournament})
function_return = to_change.apply(get_right_israel, axis=1)
df.loc[function_return.index, 'tournament'] = function_return.tournament

## Australia
to_change = df[(df.home_team=='Australia') & (df.tournament == "FIFA World Cup qualification: Asia")]
def get_right_australia(row):
    if row.year in range(1966, 2007):
        tournament = "FIFA World Cup qualification: Oceania"
    else:
        tournament = row.tournament

    return pd.Series({'tournament':tournament})
function_return = to_change.apply(get_right_australia, axis=1)
df.loc[function_return.index, 'tournament'] = function_return.tournament

to_change = df[(df.home_team=='New Zealand') & (df.tournament == "FIFA World Cup qualification: Oceania")]
def get_right_new_zealand(row):
    if row.year in range(1964,1967):
        tournament = "FIFA World Cup qualification: Oceania"
    else:
        tournament = row.tournament

    return pd.Series({'tournament':tournament})
function_return = to_change.apply(get_right_new_zealand, axis=1)
df.loc[function_return.index, 'tournament'] = function_return.tournament


# Cazaquistão:
to_change =  df[(df.home_team=='Kazakhstan') & (df.tournament == "FIFA World Cup qualification: Europe")]
def get_right_kazakhsthan(row):
    if row.year in range(1993,2003):
        tournament = "FIFA World Cup qualification: Asia"
    else:
        tournament = row.tournament

    return pd.Series({'tournament':tournament})
function_return = to_change.apply(get_right_kazakhsthan, axis=1)
df.loc[function_return.index, 'tournament'] = function_return.tournament

In [105]:
# Calcula o peso K de cada partida
criteria = {
    'k60':['FIFA World Cup'],
    'k50':['Copa América', 'UEFA Euro', 'Confederations Cup', 
           'FIFA World Cup qualification: Europe', 'FIFA World Cup qualification: South America', 'Olympic Games'],
    'k30':['African Cup of Nations', 'AFC Asian Cup', 'CONCACAF Championship',
          'Gold Cup', 'Pan American Championship', 'UEFA Euro qualification'],
    'k20':['FIFA World Cup qualification: Asia', 'FIFA World Cup qualification: North and Central America',
          'FIFA World Cup qualification: Africa', 'FIFA World Cup qualification: Oceania'],
    'k15':['African Cup of Nations qualification', 'AFC Asian Cup qualification',
          'CONCACAF Championship qualification', 'Gold Cup qualification', 
           'Oceania Nations Cup',],

}

def get_k_factor(row, criteria):
    if row.tournament in criteria['k60']:
        k_factor = 60
    elif row.tournament in criteria['k50']:
        k_factor = 50
    elif row.tournament in criteria['k30']:
        k_factor = 30
    elif row.tournament in criteria['k20']:
        k_factor = 20
    elif row.tournament in criteria['k15']:
        k_factor = 15
    else:
        k_factor = 10
    return pd.Series({'k_factor':k_factor})

# Roda função
df['k_factor'] = df.apply(get_k_factor, args=(criteria,), axis=1)

In [106]:
# Ordena partidas de acordo com data. Reseta o index na ordem cronológica
df = df.sort_values(by='datetime').reset_index(drop=True)

In [107]:
# Adiciona um id único para cada partida, com base no index + 1
df["match_id"] = [str(item) for item in (df.index.values + 1)]

## Cálculo

In [108]:
elo_df = []
elo_by_team = {}
team_list = []
seasons = df.year.unique()
for season in seasons:
    # Itera por todos os anos que o time jogou, calculando jogos falsos para implementar o decréscimo de 10% na pontuação anual 
    for team in team_list: # Note que team_list está vazia no começo do loop, então na primeira iteração,
                           # o bloco de código abaixo não é executado.
                           # Na sequência, itens são adicionados a lista e, progressivamente, 
                           # são criados os jogos fake necessários
        
        # Adiciona um jogo fake no dia primeiro de janeiro
        original_elo = elo_by_team[team][-1]['current_elo']
        decreased_elo = original_elo
        fake_date = datetime.strptime(str(season-1) + '-12-31', "%Y-%m-%d")

        # Adiciona linha no dicionário controle
        elo_by_team[team].append({'current_elo':decreased_elo, 'datetime':fake_date})

        # Adiciona uma linha no dataframe também
        elo_df.append({"datetime":fake_date, "team":team, "opponent":'end-of-year', 'opponent_elo':'',
           "goals_scored":'', "goals_conceded":'', 
           "new_elo":decreased_elo, "old_elo":original_elo,
           "location":'', 'year':season-1, 'tournament':'', 'fake_flag':'yes',
            "match_id":"0"
          })

        # Adiciona um jogo fake no dia 31 de dezembro
        
        # Calcula campos falsos
        decreased_elo = round(original_elo * 0.9)
        fake_date = datetime.strptime(str(season) + '-01-01', "%Y-%m-%d")

        # Adiciona linha no dicionário controle
        elo_by_team[team].append({'current_elo':decreased_elo, 'datetime':fake_date})

        # Adiciona uma linha no dataframe também
        elo_df.append({"datetime":fake_date, "team":team, "opponent":'beginning-of-year', 'opponent_elo':'',
           "goals_scored":'', "goals_conceded":'', 
           "new_elo":decreased_elo, "old_elo":original_elo,
           "location":'', 'year':season, 'tournament':'','fake_flag':'yes',
           "match_id":"0"
        })

    # Aqui começa o cálculo real
    for index, row in df[df.year==season].iterrows():
        home_team = row.home_team
        home_score = row.home_score
        away_team = row.away_team
        away_score = row.away_score
        difference = row.difference
        location = row.city + ', ' + row.country
        k_factor = row.k_factor
        tournament = row.tournament
        match_id = row.match_id

        # Pega o elo do time da casa
        if home_team in team_list: # Se ele já apareceu antes
            home_team_elo = elo_by_team[home_team][-1]['current_elo'] # Acessa último elo
        else: # Se ele nunca apareceu antes
            home_team_elo = 0
            team_list.append(home_team)
            elo_by_team[home_team] = []

        # Pega o elo do time visitante
        if away_team in team_list: # Se ele já apareceu antes
            away_team_elo = elo_by_team[away_team][-1]['current_elo'] # Acessa último elo
        else: # Se ele nunca apareceu antes
            away_team_elo = 0
            team_list.append(away_team)
            elo_by_team[away_team] = []

        # Calcula as chances de vitória de cada time
        chances = get_win_expectancy(home_team_elo, away_team_elo) # Chances do home team sempre na posição 0, away team na posição 1

        # Com o resultado em mãos, calcula a variação da pontuação elo das equipes
        if row.winner == 'home_team':
            home_new_elo = get_home_new_elo(1, chances, difference, k_factor, home_team_elo)
            away_new_elo = get_away_new_elo(0, chances, difference, k_factor, away_team_elo)
        elif row.winner == 'away_team':
            home_new_elo = get_home_new_elo(0, chances, difference, k_factor, home_team_elo)
            away_new_elo = get_away_new_elo(1, chances, difference, k_factor, away_team_elo)
        else: #It's a draw
            home_new_elo = get_home_new_elo(.5, chances, difference, k_factor, home_team_elo)
            away_new_elo = get_away_new_elo(.5, chances, difference, k_factor, away_team_elo)

        # Adiciona novo elo para o df
        
        # Time da casa
        elo_df.append({"datetime":row.datetime, "team":home_team, "opponent":away_team, 'opponent_elo':away_team_elo,
                       "goals_scored":home_score, "goals_conceded":away_score, 
                       "new_elo":home_new_elo, "old_elo":home_team_elo,
                       "location":location, 'year':season, 'tournament':tournament, 'fake_flag':'no',
                       "match_id":match_id
                      })

        # Time visitante
        elo_df.append({"datetime":row.datetime, "team":away_team, "opponent":home_team, 'opponent_elo':home_team_elo,
                       "goals_scored":away_score, "goals_conceded":home_score, 
                       "new_elo":away_new_elo, "old_elo":away_team_elo,
                       "location":location, 'year':season, 'tournament':tournament, 'fake_flag':'no',
                       "match_id":match_id
                      })

        # Adiciona para o dicionário de cada clube para controlar o elo anterior
        elo_by_team[home_team].append({'current_elo':home_new_elo, 'datetime':row.datetime})
        elo_by_team[away_team].append({'current_elo':away_new_elo, 'datetime':row.datetime})

In [109]:
# Salva como dataframe
elo_df = pd.DataFrame(elo_df)

In [110]:
# Coloca hífen no nome dos países
elo_df['team'] = elo_df.team.str.replace(" ","-")
elo_df['opponent'] = elo_df.opponent.str.replace(" ","-")

In [111]:
# Reordena com base na data da partida
elo_df = elo_df.sort_values(by='datetime')

# Descontinua as linhas de países extintos
- No estado atual da base de dados, os times de países extintos (URSS, Iugoslávia, Alemanha Oriental, etc.) continuam sendo atualizados todos os anos, devido à redução anual prevista no código. Vamos removê-los manualmente.

In [112]:
# Se o país for tal, remova todos os jogos falsos após o último jogo verdadeiro
extinct_teams = [
                 'Czechoslovakia',
                 'German-DR',
                 'Ireland',
                 'Mandatory-Palestine',
                 'Netherlands-Antilles',
                 'Saarland',
                 'Serbia-and-Montenegro',
                 'Soviet-Union',
                 'United-Arab-Republic',
                 'Vietnam-Republic',
                 'Yemen-AR',
                 'Yemen-DPR',
                 'Yugoslavia',
                 #North-Vietnam não está aqui porque foi removido anteriormente
                ]


def clean_extinct_teams(extinct_teams, elo_df):
    # Para cada time extinto...
    for extinct_team in extinct_teams:
        # ...selecione todos os jogos do time
        temp = elo_df[elo_df.team == extinct_team]
        temp = temp.sort_values(by='datetime', ascending=False)

        # Enquanto as partidas forem falsas, vá deletando as linhas.
        # Quando a partida for verdadeira, pare de deletar
        for index, row in temp.iterrows():
            #print(temp.loc[index])
            elo_df.drop(index, inplace=True)
            if row.fake_flag == "no":
                break

    return elo_df

In [113]:
elo_df = clean_extinct_teams(extinct_teams, elo_df)

# Converte o nome dos países usando um dicionário de códigos FIFA

In [114]:
codes = {
"Afghanistan":"AFG",
"Albania":"ALB",
"Algeria":"ALG",
"American-Samoa":"ASA",
"Andorra":"AND",
"Angola":"ANG",
"Anguilla":"AIA",
"Antigua-and-Barbuda":"ATG",
"Argentina":"ARG",
"Armenia":"ARM",
"Aruba":"ARU",
"Australia":"AUS",
"Austria":"AUT",
"Azerbaijan":"AZE",
"Bahamas":"BAH",
"Bahrain":"BHR",
"Bangladesh":"BAN",
"Barbados":"BRB",
"Belarus":"BLR",
"Belgium":"BEL",
"Belize":"BLZ",
"Benin":"BEN",
"Bermuda":"BER",
"Bhutan":"BHU",
"Bolivia":"BOL",
"Bosnia-Herzegovina":"BIH",
"Botswana":"BOT",
"Brazil":"BRA",
"British-Virgin-Islands":"VGB",
"Brunei":"BRU",
"Bulgaria":"BUL",
"Burkina-Faso":"BFA",
"Burundi":"BDI",
"Cambodia":"CAM",
"Cameroon":"CMR",
"Canada":"CAN",
"Cape-Verde-Islands":"CPV",
"Cayman-Islands":"CAY",
"Central-African-Republic":"CTA",
"Chad":"CHA",
"Chile":"CHI",
"China":"CHN",
"Chinese-Taipei":"TPE",
"Colombia":"COL",
"Comoros":"COM",
"Congo":"CGO",
"Cook-Islands":"COK",
"Costa-Rica":"CRC",
"Croatia":"CRO",
"Cuba":"CUB",
"Curaçao":"CUW",
"Cyprus":"CYP",
"Czech-Republic":"CZE",
"Denmark":"DEN",
"Djibouti":"DJI",
"Dominica":"DMA",
"Dominican-Republic":"DOM",
"Congo-DR":"COD",
"Ecuador":"ECU",
"Egypt":"EGY",
"El-Salvador":"SLV",
"England":"ENG",
"Equatorial-Guinea":"EQG",
"Eritrea":"ERI",
"Estonia":"EST",
"Ethiopia":"ETH",
"Faroe-Islands":"FRO",
"Fiji":"FIJ",
"Finland":"FIN",
"France":"FRA",
"Gabon":"GAB",
"Gambia":"GAM",
"Georgia":"GEO",
"Germany":"GER",
"German-DR":"DDR",
"Ghana":"GHA",
"Gibraltar":"GIB",
"Greece":"GRE",
"Grenada":"GRN",
"Guam":"GUM",
"Guatemala":"GUA",
"Guinea":"GUI",
"Guinea-Bissau":"GNB",
"Guyana":"GUY",
"Haiti":"HAI",
"Honduras":"HON",
"Hong-Kong":"HKG",
"Hungary":"HUN",
"Iceland":"ISL",
"India":"IND",
"Indonesia":"IDN",
"Iran":"IRN",
"Iraq":"IRQ",
"Israel":"ISR",
"Italy":"ITA",
"Ivory-Coast":"CIV",
"Jamaica":"JAM",
"Japan":"JPN",
"Jordan":"JOR",
"Kazakhstan":"KAZ",
"Kenya":"KEN",
"Kosovo":"KVX",
"Kuwait":"KUW",
"Kyrgyzstan":"KGZ",
"Laos":"LAO",
"Latvia":"LVA",
"Lebanon":"LIB",
"Korea-Republic":"KOR",
"Korea-DPR":"PRK",
"Laos":"LAO",
"Latvia":"LVA",
"Lebanon":"LIB",
"Lesotho":"LES",
"Liberia":"LBR",
"Libya":"LBY",
"Liechtenstein":"LIE",
"Lithuania":"LTU",
"Luxembourg":"LUX",
"Macau":"MAC",
"Macedonia":"MKD",
"Madagascar":"MAD",
"Malawi":"MWI",
"Malaysia":"MAS",
"Maldives":"MDV",
"Mali":"MLI",
"Malta":"MLT",
"Mauritania":"MTN",
"Mauritius":"MRI",
"Mexico":"MEX",
"Moldova":"MDA",
"Mongolia":"MNG",
"Montenegro":"MNE",
"Montserrat":"MSR",
"Morocco":"MAR",
"Mozambique":"MOZ",
"Myanmar":"MYA",
"Namibia":"NAM",
"Nepal":"NEP",
"Netherlands":"NED",
"New-Caledonia":"NCL",
"New-Zealand":"NZL",
"Nicaragua":"NCA",
"Niger":"NIG",
"Nigeria":"NGA",
"Northern-Ireland":"NIR",
"Norway":"NOR",
"Oman":"OMA",
"Pakistan":"PAK",
"Palestine":"PLE",
"Panama":"PAN",
"Papua-New-Guinea":"PNG",
"Paraguay":"PAR",
"Peru":"PER",
"Philippines":"PHI",
"Poland":"POL",
"Portugal":"POR",
"Puerto-Rico":"PUR",
"Qatar":"QAT",
"Republic-of-Ireland":"IRL",
"Romania":"ROU",
"Russia":"RUS",
"Rwanda":"RWA",
"St-Kitts-and-Nevis":"SKN",
"St-Lucia":"LCA",
"St-Vincent-and-the-Grenadines":"VIN",
"Samoa":"SAM",
"San-Marino":"SMR",
"São-Tomé-and-Príncipe":"STP",
"Saudi-Arabia":"KSA",
"Scotland":"SCO",
"Senegal":"SEN",
"Serbia":"SRB",
"Seychelles":"SEY",
"Sierra-Leone":"SLE",
"Singapore":"SIN",
"Slovakia":"SVK",
"Slovenia":"SVN",
"Solomon-Islands":"SOL",
"Somalia":"SOM",
"South-Africa":"RSA",
"Spain":"ESP",
"Sri-Lanka":"SRI",
"Sudan":"SDN",
"South-Sudan":"SSD",
"Suriname":"SUR",
"Swaziland":"SWZ",
"Sweden":"SWE",
"Switzerland":"SUI",
"Syria":"SYR",
"Tahiti":"TAH",
"Tajikistan":"TJK",
"Tanzania":"TAN",
"Thailand":"THA",
"East-Timor":"TLS",
"Togo":"TOG",
"Tonga":"TGA",
"Trinidad-and-Tobago":"TRI",
"Tunisia":"TUN",
"Turkey":"TUR",
"Turkmenistan":"TKM",
"Turks-and-Caicos-Islands":"TCA",
"Uganda":"UGA",
"Ukraine":"UKR",
"United-Arab-Emirates":"UAE",
"United-States":"USA",
"Uruguay":"URU",
"Virgin-Islands":"VIR",
"Uzbekistan":"UZB",
"Vanuatu":"VAN",
"Venezuela":"VEN",
"Vietnam":"VIE",
"Vietnam-Republic":"VSO",
"Wales":"WAL",
"Yemen":"YEM",
"Zambia":"ZAM",
"Zimbabwe":"ZIM",
"Bonaire":"BOE",
"French-Guiana":"GUF",
"Great-Britain":"GBR",
"Guadeloupe":"GLP",
"Martinique":"MTQ",
"Réunion":"REU",
"St-Martin":"SMN",
"Sint-Maarten":"SMA",
"Tuvalu":"TUV",
"Zanzibar":"ZAN",
"Micronesia-FS":"FSM",
"Kiribati":"KIR",
"Niue":"NIU",
"Northern-Mariana-Islands":"NMI",
"Palau":"PLW",
"Aden":"ADE",
"British-Guiana":"BGU",
"British-Honduras":"BHO",
"British-India":"BIN",
"Bohemia":"BOH",
"Burma":"BUR",
"Central-African-Republic":"CAF",
"Ceylon":"CEY",
"Commonwealth-of-Independent-States":"CIS",
"Congo-Kinshasa":"CKN",
"Congo-Brazzaville":"COB",
"Czechoslovakia":"TCH",
"Dahomey":"DAH",
"Dutch-East-Indies":"DEI",
"East-Germany":"DDR",
"Gold-Coast":"GOC",
"Ireland":"EIR",
"Malaya":"MWI",
"Netherlands-Antilles":"ANT",
"Netherlands":"HOL",
"New-Hebrides":"HEB",
"North-Borneo":"NBO",
"North-Vietnam":"VNO",
"Yemen-AR":"NYE",
"Northern-Rhodesia":"NRH",
"Mandatory-Palestine":"PAL",
"Romania":"ROM",
"Rhodesia":"RHO",
"Saar":"SAA",
"Serbia-and-Montenegro":"SCG",
"Siam":"SIA",
"Southern-Rhodesia":"SRH",
"South-Vietnam":"VSO",
"Yemen-DPR":"SYE",
"Soviet-Union":"URS",
"Sudan":"SUD",
"Suriname":"NGY",
"Tanganyika":"TAA",
"Taiwan":"TAI",
"United-Arab-Republic":"UAR",
"Upper-Volta":"UPV",
"West-Germany":"FRG",
"Western-Samoa":"WSM",
"Yugoslavia":"YUG",
"Zaire":"ZAI", 
"Saarland":"SAA",    
}

In [115]:
%%time
def get_country_codes(row, codes):
    for k,v in codes.items():        
        if row.team == k:
            team = v
            break
            
        else:
            team = row.team
            
    for k,v in codes.items():        
        if row.opponent == k:
            opponent = v
            break
            
        else:
            opponent = row.opponent
            
            
    return pd.Series({"team":team, "opponent":opponent})
    
function_return = elo_df.apply(get_country_codes, args=(codes,), axis=1)
elo_df[['team', 'opponent']] = function_return[['team', 'opponent']]

CPU times: user 7min 13s, sys: 668 ms, total: 7min 13s
Wall time: 7min 15s


In [116]:
elo_df['datetime']

0       1872-11-30
1       1872-11-30
2       1872-12-31
4       1872-12-31
3       1873-01-01
5       1873-01-01
6       1873-03-08
7       1873-03-08
8       1873-12-31
10      1873-12-31
9       1874-01-01
11      1874-01-01
13      1874-03-07
12      1874-03-07
14      1874-12-31
16      1874-12-31
15      1875-01-01
17      1875-01-01
18      1875-03-06
19      1875-03-06
20      1875-12-31
22      1875-12-31
21      1876-01-01
23      1876-01-01
25      1876-03-04
24      1876-03-04
26      1876-03-25
27      1876-03-25
28      1876-12-31
30      1876-12-31
32      1876-12-31
29      1877-01-01
31      1877-01-01
33      1877-01-01
34      1877-03-03
35      1877-03-03
37      1877-03-05
36      1877-03-05
38      1877-12-31
40      1877-12-31
42      1877-12-31
39      1878-01-01
41      1878-01-01
43      1878-01-01
44      1878-03-02
45      1878-03-02
46      1878-03-23
47      1878-03-23
52      1878-12-31
50      1878-12-31
48      1878-12-31
51      1879-01-01
49      1879

# Salva pra csv

In [117]:
string = str(datetime.now())
string = string[:10]
string

'2018-07-16'

In [118]:
elo_df.to_csv('../data/elo-selecoes-' + string + '.csv', index=False)